In [1]:
using Pkg
Pkg.status("JuMP")

    Status `C:\Users\BV SAMEER KUMAR\.julia\environments\v1.2\Project.toml`
  [4076af6c] JuMP v0.18.6


In [2]:
using JuMP
using Gurobi
# Linear Algebra packages
using LinearAlgebra
using SparseArrays

In [3]:
function problem_4_model(t, P1, P2, P3, P4, y1, y2, y3, y4, env)

    model = Model(solver = GurobiSolver(env, OutputFlag = 0));

    @variable(model, x[1:3]);
    @variable(model, v1[1:3]);
    @variable(model, v2[1:3]);
    @variable(model, v3[1:3]);
    @variable(model, v4[1:3]);

    @constraint(model, v1 .== P1 * [x; 1]);
    @constraint(model, v2 .== P2 * [x; 1]);
    @constraint(model, v3 .== P3 * [x; 1]);
    @constraint(model, v4 .== P4 * [x; 1]);

    @constraint(model, norm( v1[1:2] - v1[3] * y1) <= t * v1[3])
    @constraint(model, norm( v2[1:2] - v2[3] * y2) <= t * v2[3])
    @constraint(model, norm( v3[1:2] - v3[3] * y3) <= t * v3[3])
    @constraint(model, norm( v4[1:2] - v4[3] * y4) <= t * v4[3])

    @objective(model, Min, t)

    return model

end

gurobi_env = Gurobi.Env();

P1 = [1 0 0 0; 0 1 0 0; 0 0 1 0];
P2 = [1 0 0 0; 0 0 1 0; 0 -1 0 10];
P3 = [1 1 1 -10; -1 1 1 0; -1 -1 1 10];
P4 = [0 1 1 0; 0 -1 1 0; -1 0 0 10];

y1 = [0.98; 0.93];
y2 = [1.01; 1.01];
y3 = [0.95; 1.05];
y4 = [2.04; 0.00];

lower = 0;
upper = 1;

tol = 1e-5;
lastx = 0;
while (upper-lower) > tol
    t = 0.5 * (lower + upper);
    model = problem_4_model(t, P1, P2, P3, P4, y1, y2, y3, y4, gurobi_env)
    status = solve(model);

    optx = getvalue(model[:x]);
    model = nothing

    if isnan(optx[1]) || isnan(optx[2]) || isnan(optx[3])
        lower = t
    else
        lastx = optx;
        upper = t;
    end
end
println("Optimal Estimate of X: ", lastx)

Academic license - for non-commercial use only


┌ Warning: Gurobi reported infeasible or unbounded. Set InfUnbdInfo=1 for more specific status.
└ @ Gurobi C:\Users\BV SAMEER KUMAR\.julia\packages\Gurobi\TX8tY\src\MPB_wrapper.jl:438
┌ Warning: Not solved to optimality, status: InfeasibleOrUnbounded
└ @ JuMP C:\Users\BV SAMEER KUMAR\.julia\packages\JuMP\I7whV\src\solvers.jl:212
┌ Warning: Variable value not defined for component of x. Check that the model was properly solved.
└ @ JuMP C:\Users\BV SAMEER KUMAR\.julia\packages\JuMP\I7whV\src\JuMP.jl:480
┌ Warning: Gurobi reported infeasible or unbounded. Set InfUnbdInfo=1 for more specific status.
└ @ Gurobi C:\Users\BV SAMEER KUMAR\.julia\packages\Gurobi\TX8tY\src\MPB_wrapper.jl:438
┌ Warning: Not solved to optimality, status: InfeasibleOrUnbounded
└ @ JuMP C:\Users\BV SAMEER KUMAR\.julia\packages\JuMP\I7whV\src\solvers.jl:212
┌ Warning: Variable value not defined for component of x. Check that the model was properly solved.
└ @ JuMP C:\Users\BV SAMEER KUMAR\.julia\packages\JuMP\I7whV\

Optimal Estimate of X: [4.911839825574044, 5.015628432799912, 5.197602450915133]
